In [1]:
import sopa
import spatialdata
import pandas as pd
import scanpy as sc
import numpy as np

/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
sdata = spatialdata.read_zarr("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V5.zarr/")
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creati

SpatialData object, with associated Zarr store: /Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V5.zarr
├── Images
│     ├── 'HE': DataTree[cyx] (3, 6926, 21997), (3, 3464, 10999), (3, 1731, 5500), (3, 866, 2750), (3, 433, 1375), (3, 216, 687)
│     ├── 'PhenoCycler_Image': DataTree[cyx] (25, 6391, 20981), (25, 3195, 10490), (25, 1597, 5245), (25, 798, 2622), (25, 399, 1311)
│     └── 'Xenium_Image': DataTree[cyx] (5, 17141, 51168), (5, 8570, 25584), (5, 4285, 12792), (5, 2142, 6396), (5, 1071, 3198)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (17141, 51168), (8570, 25584), (4285, 12792), (2142, 6396), (1071, 3198)
│     └── 'nucleus_labels': DataTree[yx] (17141, 51168), (8570, 25584), (4285, 12792), (2142, 6396), (1071, 3198)
├── Points
│     ├── 'XeniumLnmk': DataFrame with shape: (<Delayed>, 2) (2D points)
│     ├── 'Xenium_Lnmks': DataFrame with shape: (<Delayed>, 2) (2D points)
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── S

In [14]:
import pandas as pd

shape     = sdata.shapes['cell_boundaries']
new_ids   = sdata.tables["GeneTranscripts_XStock"].obs["cell_id"].tolist()

records = []
# zip the new_ids to the geometry items (ensure they’re in the same order!)
for new_id, (old_id, poly) in zip(new_ids, shape['geometry'].items()):
    for vi, (x, y) in enumerate(poly.exterior.coords, start=1):
        records.append({
            'cell_id': new_id,
            'vertex':  vi,
            'x':       x,
            'y':       y
        })

shape_df = pd.DataFrame(records)
shape_df

,cell_id,vertex,x,y
0,aaaacnhm-1,1,302.175018,2457.562500
1,aaaacnhm-1,2,301.112518,2457.987549
2,aaaacnhm-1,3,299.837494,2459.262451
3,aaaacnhm-1,4,299.625000,2459.687500
4,aaaacnhm-1,5,299.625000,2460.750000
...,...,...,...,...
2829499,oiipfnij-1,21,9014.038086,1687.887573
2829500,oiipfnij-1,22,9013.612305,1687.887573
2829501,oiipfnij-1,23,9012.975586,1687.462524
2829502,oiipfnij-1,24,9012.762695,1686.612549


In [15]:
#We just need it to obtain shape of image
C=0
img = sdata.images['Xenium_Image']['scale0']['image'].isel(c=C)
arr    = np.asarray(img)


import numpy as np
from skimage.draw import polygon

#Fill with cell_id

H, W = arr.shape
mask = np.zeros((H, W), dtype=np.int32)

proportion_constant = 4.7065

labels = {}
for label, (cell_id, grp) in enumerate(shape_df.groupby('cell_id'), start=1):
    labels[label] = cell_id
    cols = np.floor(grp['x'].values * proportion_constant).astype(int)
    rows = np.floor(grp['y'].values * proportion_constant).astype(int)

    # clip into valid pixel bounds
    cols = np.clip(cols, 0, W-1)
    rows = np.clip(rows, 0, H-1)

    # rasterize the polygon, filling those pixels with `label`
    rr, cc = polygon(rows, cols, shape=mask.shape)
    mask[rr, cc] = label


In [16]:
np.savetxt("Tissue1_mask.csv", mask, fmt="%d", delimiter=",")
records_df = pd.DataFrame(records)
records_df.to_csv("Tissue1_mask_labels.csv", header=False, index=False)

## Mask is prepared (the matrix defined the borders based on index), and it must be applied props function after that point.

In [ ]:
import numpy as np
import pandas as pd
from skimage import data, filters, measure, morphology
import plotly.express as px
import plotly.graph_objects as go
import tifffile as tfi


for i in range(0, 4):
    try:
        img_raw = sdata.images['Xenium_Image']['scale0']['image'].isel(c=i)
        #label_image = np.loadtxt("/Users/ugursahin/RAO/Tissue1_mask.csv", delimiter=",", dtype=int)
        label_image = mask
        
        img = np.asarray(img_raw)

        props = measure.regionprops_table(
            label_image,
            intensity_image=img,
            properties=['area',
        'area_bbox',
        'area_convex',
        'area_filled',
        'axis_major_length',
        'axis_minor_length',
        'centroid',
        'centroid_local',
        'centroid_weighted',
        'centroid_weighted_local',
        'coords',
        'eccentricity',
        'equivalent_diameter_area',
        'euler_number',
        'extent',
        'feret_diameter_max',
        'image',
        'image_convex',
        'image_filled',
        'image_intensity',
        'inertia_tensor',
        'inertia_tensor_eigvals',
        'intensity_max',
        'intensity_mean',
        'intensity_min',
        'label',
        'moments',
        'moments_central',
        'moments_hu',
        'moments_normalized',
        'moments_weighted',
        'moments_weighted_central',
        'moments_weighted_hu',
        'moments_weighted_normalized',
        'orientation',
        'perimeter',
        'perimeter_crofton',
        'slice',
        'solidity']
        )


        props_df = pd.DataFrame(props)
        props_df.to_csv("Region_1_ExtractedProps"+"_forChannel"+str(i)+".csv", header=True)
    except:
        print("Error in processing image for channel: ", i)
        continue


Error in processing image for channel:  0
Error in processing image for channel:  1
